In [161]:
import os
import pandas as pd
import re

Now that we are in the preferable working directory, I'll explore each data file's variables in order to figure out how to join each dataframe into a big dataset. Let's begin chronologically:

# 2000-2020

In [5]:
df_2000=pd.read_csv(f"{data_path}/2000.csv")
df_2005=pd.read_csv(f"{data_path}/2005.csv")
df_2010=pd.read_csv(f"{data_path}/2010.csv")
df_2020=pd.read_csv(f"{data_path}/2020.csv")

/var/folders/m5/26l5nfs54r94_664jm4k6zvr0000gn/T/ipykernel_9866/36002156.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020=pd.read_csv(f"{data_path}/2020.csv")


I think it will be a good idea to split the population columns into categories. For example, the 'Población de 15 años y más' column will be abreviated as 'POB_15+', and each description will be abreviated as well.

I'll begin with abreviating the first part of the population columns.









Now, I need to find a good use of regular expressions in order to speed things up. I want for the next columns to have the name structure: "POB_[age_range]_[category]". 

In [407]:

def population(column_name):
    match=re.search("Población",column_name)
    if match:
        sex=re.search("femenina|masculina",column_name)
        if sex:
            letter=str.upper(sex.group()[0])
            return f"POB {letter}"
        return "POB"
    
def age_range(column_name):
    """ 
    
    Takes a column name as an input and spits out the age range the input contains
    
    """
    # Case for specified age range:
    age_range_pattern=r'\d+ a \d+' 
    match=re.search(age_range_pattern,column_name)
    if match:
        age_range=match.group()
        #age_range=str.replace(age_range," ","_")
        return age_range
    
    # Case for ambiguos age range:
    ambiguos_age_range_pattern_1=r"\d+ años y más"
    match=re.search(ambiguos_age_range_pattern_1,column_name)
    if match:
        text=match.group(0)
        age_range=f"{re.findall(r'\d+',column_name)[0]}+"
        return age_range
    
    ambiguos_age_range_pattern_2=r"\d+ años"
    match=re.search(ambiguos_age_range_pattern_2,column_name)
    if match:
        text=match.group(0)
        age_range=f"{re.findall(r'\d+',column_name)[0]}+"
        return age_range   
    
    
def category(column_name):
    """ 
    
    Takes a column name as an input and spits out the category wanted depending on what the input contains
    
    """

    categories={
         #Education
        "que no asiste a la escuela":"NAE",
        "que asiste a la escuela":"AE",
        

        "que sabe leer y escribir":"SLE",
        "que saben leer y escribir":"SLE",
        "que no sabe leer y escribir":"NSLE",
        "que no saben leer y escribir":"NSLE",
        " alfabeta":"ABT",
        "analfabeta":"ANBT",

        "sin escolaridad":"SIN_E",
        "primaria completa":"PRIM_COM",
        "con instrucción media superior":"PREPA",
        "con instrucción superior":"CARRERA",
        "con instrucción posprimaria":"POS_PRIM",
        "sin instrucción posprimaria":"SIN_POS_PRIM",
        "sin instrucción":"KINDER",
        "primaria incompleta":"PRIM_INC",
        "secundaria completa":"SEC_COM",
        "educación básica completa":"SEC_COM",
        "secundaria incompleta":"SEC_INC",
        "educación básica incompleta":"SEC_INC",
        "educación pos-básica":"POSBASICA",
        "educación posbásica":"POSBASICA",
        
        
        
        #Financial
        "más económicamente activa":"EC_ACTIVA",
        "no económicamente activa":"NO_EC_ACTIVA",
        " ocupada":"EMPLEADA",
        "desocupada":"DESEMPLEADA"


    }
    for replacement,category in categories.items():
        match=re.search(fr'{replacement}',column_name)
        if match:
            return category
    #print("No matches :( ")


def grade(column_name):
        match=re.search("Grado promedio",column_name)
        if match:
            sex=re.search("femenina|masculina",column_name)
            if sex:
                letter=str.upper(sex.group()[0])
                return f"GDO {letter}"
            return "GDO"
        

def other(column_name):
        others={
        #2020
         'Clave de entidad federativa':'CVE_ENT',
         'Entidad federativa':'ENT',
         'Clave de municipio o demarcación territorial':'CVE TERR',
         'Municipio o demarcación territorial':'MUN',
         'Clave de localidad':'CVE_LOC',
         'Clave de Localidad':'CVE_LOC',
         'Localidad':'LOCALIDAD',
         'Longitud':'LON',
         'Latitud':'LAT',
         'Altitud':'ALTI',
         'Población total':'POB_TOT',
         
        #2010
        'Clave de municipio ó delegación':'CVE TERR',
        'Municipio ó delegación':'MUN',

        #2005
        'Entidad Federativa':'ENT',
        
        'Clave del Municipio':'CVE TERR',
        'Municipio':'MUN',

        #2000
        'Nombre de la entidad':'ENT',
        'Clave de municipio o delegación':'CVE TERR',
        'Nombre del municipio o delegación':'MUN',
        'Nombre de la localidad':'LOCALIDAD',

        # Exceptions
        'Población de cero a 14 años':'POB_0 a 14',
        'Po blación femenina de 15 a 49 años':'POB_15 a 49'
          }
        


        for replacement,other in others.items():
            match=re.search(fr'{replacement}',column_name)
            if match:
                return other
            

def rename_columns(df):
        original_columns={}
    
        for column in df.columns:

            # Other columns:
            if other(column):
                original_columns[column]=other(column)
                continue

            # Population columns
            if population(column):
                if age_range(column):
                    if category(column):
                        #print(column, f"{population(column)}_{age_range(column)}_{category(column)}")
                        original_columns[column]=f"{population(column)}_{age_range(column)}_{category(column)}"
                        continue
            
                    #print(column, f"{population(column)}_{age_range(column)}")
                    original_columns[column]=f"{population(column)}_{age_range(column)}"
                    continue

                #print(column, f"{population(column)}")
                original_columns[column]=f"{population(column)}"
                continue

            # Grade Columns
            if grade(column):
                original_columns[column]=f"{grade(column)}"
                #print(column,f"{grade(column)}")
                continue
            
        
        df=df.rename(columns=original_columns)
            
        return df,original_columns

In [242]:
rename_columns(df_2020)[0].head(3)

,Unnamed: 0,CVE_ENT,ENT,CVE TERR,MUN,CVE_LOC,LOCALIDAD,LON,LAT,ALTI,...,POB M_12+_EC_ACTIVA,POB_12+_EC_ACTIVA,POB F_12+_EC_ACTIVA,POB M_12+_EC_ACTIVA,POB_12+_EMPLEADA,POB F_12+_EMPLEADA,POB M_12+_EMPLEADA,POB_12+_DESEMPLEADA,POB F_12+_DESEMPLEADA,POB M_12+_DESEMPLEADA
0,0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,...,36815941,37891261,26379060,11512201,61121324,25137019,35984305,1160310,328674,831636
1,1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN,NaN,...,95222,68766,42827,25939,127469,32906,94563,820,161,659
2,2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,NaN,...,50429,46162,28959,17203,68215,18365,49850,689,110,579


In [404]:
def test_rename_columns(df):
    """ 
    This function takes a dataframe and checks if there are columns with the same name after renaming them using the 'rename_columns' function.
    It spits out columns that don't have a unique name after being renamed.
    """
    
    problem_original=[]
    problem_renamed=[]
    column_dictionary=rename_columns(df)[1]


    # Original Columns
    original_column_list=column_dictionary.keys()
    unique_original_columns=set(original_column_list)
    not_unique_original=len(unique_original_columns)!=len(original_column_list) 
   
    if not_unique_original:
        for column in original_column_list:
            if original_column_list.count(column) > 1 and column not in problem_original:
                problem_original.append(column)


    # Renamed Columns
    renamed_column_list=list(column_dictionary.values())
    unique_renamed_columns=set(renamed_column_list)
    not_unique_renamed=len(unique_renamed_columns)!=len(renamed_column_list)

    if not_unique_renamed:
        for original_column,renamed_column in column_dictionary.items():
            if renamed_column_list.count(renamed_column) > 1 and (original_column,renamed_column) not in problem_renamed:
                problem_renamed.append((original_column,renamed_column))

    
    return (problem_original,problem_renamed)



   


In [408]:
test_rename_columns(filter(df_2020))

([], [])

In [409]:
test_rename_columns(filter(df_2010))

([], [])

In [410]:
test_rename_columns(filter(df_2005))

([], [])

In [411]:
test_rename_columns(filter(df_2000))

([], [])

Now that I have a good function to rename my variables, all we need now is to filter the columns we aren't interested in,(again): 

In [317]:
def filter(df):
    
    columns_staying=[]
    columns_to_filter_out=[
        'Relación hombres-mujeres',
        'Población de 15 años y más con instrucción secundaria o estudios técnicos o comerciales con primaria terminada',
        # Disabilities
        'Población con discapacidad', 
        'Población con discapacidad para hablar o comunicarse', 
        'Población con discapacidad para oír, aun usando aparato auditivo', 
        'Población con discapacidad para vestirse, bañarse o comer', 
        'Población con discapacidad para recordar o concentrarse', 
        'Población con limitación', 
        'Población con limitación para caminar, subir o bajar', 
        'Población con limitación para ver, aun usando lentes', 
        'Población con limitación para hablar o comunicarse', 
        'Población con limitación para oír, aun usando aparato auditivo', 
        'Población con limitación para vestirse, bañarse o comer', 
        'Población con limitación para recordar o concentrarse', 
        'Población con algún problema o condición mental', 
        'Población sin discapacidad, limitación, problema o condición mental',

        'Población con limitación en la actividad',
        'Población con limitación para caminar o moverse, subir o bajar',
        'Población con limitación para ver, aún usando lentes',
        'Población con limitación para hablar, comunicarse o conversar',
        'Población con limitación para escuchar',
        'Población con limitación para poner atención o aprender cosas sencillas',
        'Población con limitación mental',
        'Población sin limitación en la actividad',
        # Financial
        
       'Población de 12 años y más económicamente activa',
       'Población femenina de 12 años y más económicamente activa',
       'Población masculina de 12 años y más económicamente activa',
       'Población de 12 años y más no económicamente activa',
       'Población femenina de 12 años y más no económicamente activa',
       'Población masculina de 12 años y más no económicamente activa',
       'Población de 12 años y más ocupada',
       'Población femenina de 12 años y más ocupada',
       'Población masculina de 12 años y más ocupada',
       'Población de 12 años y más desocupada',
       'Población femenina de 12 años y más desocupada',
       'Población masculina de 12 años y más desocupada'
       ]
    
    for column in df.columns:

        if column in columns_to_filter_out:
            continue

        columns_staying.append(column)

    filtered_df=df[columns_staying]
    
    return filtered_df

Now comes the hard part: concatenating these bad boys. What we can simply do is stack one dataset over the other, and fill the variables that one doesn't have with the population value of the year plus 666, to know they come from this transformation.

In [412]:
renamed_dfs=[]
dataframes=[df_2020, df_2010, df_2005, df_2000]
for df in dataframes:
    filtered_df=filter(df)
    renamed_df=rename_columns(filtered_df)[0]
    renamed_dfs.append(renamed_df)
    
                      

In [413]:
def joining(dataframes):
    """ 
    This function takes a list of dataframes, adds missing columns to each dataframe, based on the dataframe with the most columns
    """
    reference_df=max(dataframes,key=lambda df: len(df.columns))
    reference_columns=reference_df.columns
    for df in dataframes:
        for column in reference_columns:
            if column not in df.columns:
                data_type=reference_df[column].dtype
                print(data_type)
                #df[column]=
    #dataset=pd.concat()
    #return dataset



In [414]:
renamed_dfs?

Type:        list
String form:
[        Unnamed: 0  CVE_ENT             ENT  CVE TERR  \
           0                0        0  Total naci <...>               0  3.0
           112861              0                0  3.0
           
           [112862 rows x 47 columns]]
Length:      4
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.

In [416]:
joining(renamed_dfs)

TypeError: 'dict_keys' object is not callable